- id 구성과 join할 때 결측이 어떻게 될지 등을 미리 살펴봄
- 채택한 lightgbm의 경우 결측을 자체적으로 처리해줄 수 있으므로 데이터 있는 경우만 처리할 예정

In [2]:
from glob import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data_path = "../take-home-interview/"

In [4]:
app = pd.read_csv(data_path + "loan_application.csv")
credits = pd.read_csv(data_path + "defi_credits.csv")
credit_balance = pd.read_csv(data_path + "defi_credits_balance.csv")
balance = pd.read_csv(data_path + "aave_loan_balance.csv")
payments = pd.read_csv(data_path + "installments_payments.csv")
app_prev = pd.read_csv(data_path + "application_previous.csv")

In [61]:
app_prev[app_prev.application_id_current == 108129]

,application_id_previous,application_id_current,amt_annuity,amt_application,amt_credit,amt_down_payment,amt_goods_price,weekday_appr_process_start,hour_appr_process_start,flag_last_appl_per_contract,...,sellerplace_area,name_seller_industry,cnt_payment,name_yield_group,days_first_drawing,days_first_due,days_last_due_1st_version,days_last_due,days_termination,nflag_insured_on_approval
1,2802425,108129,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,...,-1,XNA,36.0,low_action,365243.0,-134.0,916.0,365243.0,365243.0,1.0
335841,1536272,108129,21709.125,450000.0,512370.0,NaN,450000.0,WEDNESDAY,9,Y,...,6,XNA,36.0,low_normal,365243.0,-485.0,565.0,-155.0,-147.0,1.0
588441,2068863,108129,4830.930,47250.0,23688.0,24750.0,47250.0,THURSDAY,11,Y,...,110,Consumer electronics,6.0,high,365243.0,-588.0,-438.0,-588.0,-580.0,0.0
617224,2551979,108129,6664.275,71352.0,71352.0,0.0,71352.0,WEDNESDAY,9,Y,...,108,Furniture,12.0,low_normal,365243.0,-1176.0,-846.0,-846.0,-840.0,0.0
692217,2517198,108129,11250.000,0.0,225000.0,NaN,NaN,TUESDAY,13,Y,...,-1,XNA,0.0,XNA,-713.0,-673.0,365243.0,-461.0,-61.0,0.0
1380202,1760610,108129,8593.965,33052.5,33052.5,0.0,33052.5,SUNDAY,10,Y,...,108,Furniture,4.0,low_action,365243.0,-783.0,-693.0,-753.0,-748.0,0.0


### make id set

In [6]:
app_id_set = set(app.application_id_current)

credit_app_id_set = set(credits.application_id_current)
credit_defi_id_set = set(credits.defi_id)

credit_balance_defi_id_set = set(credit_balance.defi_id)

balance_app_id_set = set(balance.application_id_current)
balance_app_prev_id_set = set(balance.application_id_previous)

payments_app_id_set = set(payments.application_id_current)
payments_app_prev_id_set = set(payments.application_id_previous)

app_prev_app_id_set = set(app_prev.application_id_current)
app_prev_app_prev_id_set = set(app_prev.application_id_previous)

**Understanding app id and prev app id**

In [48]:
app_id_count = len(app_id_set)
app_prev_id_count = len(app_prev_app_prev_id_set)

app_prev_inter_count = len(app_prev_app_id_set & app_id_set)

In [50]:
## app prev에 다시 app이 들어가기도 한다
print(f"app id count: {app_id_count:,}")
print(f"app prev id count: {app_prev_id_count:,}")
print(f"app and app prev inter count: {app_prev_inter_count:,}({app_prev_inter_count/app_id_count*100:.2f}%)")

app id count: 356,255
app prev id count: 1,670,214
app and app prev inter count: 338,857(95.12%)


### Intersection ratio

**app id**

In [57]:
credit_inter_count = len(app_id_set & credit_app_id_set)
if len(credit_app_id_set - app_id_set) > 0:
    print("credit에 추가적인 app id 존재")

print(f"credits 교집합: {credit_inter_count:,}({credit_inter_count/app_id_count*100:.2f}%)")

balance_inter_count = len(app_id_set & balance_app_id_set)
if len(balance_app_id_set - app_id_set) > 0:
    print("balance에 추가적인 app id 존재")

print(f"balance 교집합: {balance_inter_count:,}({balance_inter_count/app_id_count*100:.2f}%)")

payments_inter_count = len(app_id_set & payments_app_id_set)
if len(payments_app_id_set - app_id_set) > 0:
    print("payments에 추가적인 app id 존재")

print(f"payments 교집합: {payments_inter_count:,}({payments_inter_count/app_id_count*100:.2f}%)")

app_prev_inter_count = len(app_id_set & app_prev_app_id_set)
if len(app_prev_app_id_set - app_id_set) > 0:
    print("app_prev에 추가적인 app id 존재")

print(f"app_prev 교집합: {app_prev_inter_count:,}({app_prev_inter_count/app_id_count*100:.2f}%)")

credits 교집합: 305,811(85.84%)
balance 교집합: 103,558(29.07%)
payments 교집합: 339,587(95.32%)
app_prev 교집합: 338,857(95.12%)


**app id and prev**

In [56]:
## balance나 payments에 있는 prev에 현재 app이 들어가지는 않는다
balance_prev_inter_count = len(app_id_set & balance_app_prev_id_set)
print(f"balance prev 교집합: {balance_prev_inter_count:,}({balance_prev_inter_count/app_id_count*100:.2f}%)")

payments_prev_inter_count = len(app_id_set & payments_app_prev_id_set)
print(f"payments prev 교집합: {payments_prev_inter_count:,}({payments_prev_inter_count/app_id_count*100:.2f}%)")

balance prev 교집합: 0(0.00%)
payments prev 교집합: 0(0.00%)


**prev id**

In [59]:
len(balance_app_prev_id_set - app_prev_app_prev_id_set)

11372

In [71]:
balance_inter_count = len(app_prev_app_prev_id_set & balance_app_prev_id_set)
balance_extra_prev_id_count = len(balance_app_prev_id_set - app_prev_app_prev_id_set)
if balance_extra_prev_id_count > 0:
    print(f"balance 추가적인 app prev id 존재: {balance_extra_prev_id_count:,}")

print(f"balance prev id 교집합: {balance_inter_count:,}({balance_inter_count/app_prev_id_count*100:.2f}%)")

balance 추가적인 app prev id 존재: 11,372
balance prev id 교집합: 92,935(5.56%)


In [72]:
payments_inter_count = len(app_prev_app_prev_id_set & payments_app_prev_id_set)
payments_extra_prev_id_count = len(payments_app_prev_id_set - app_prev_app_prev_id_set)
if payments_extra_prev_id_count > 0:
    print(f"payments 추가적인 app prev id 존재: {payments_extra_prev_id_count:,}")

print(f"payments prev id 교집합: {payments_inter_count:,}({payments_inter_count/app_prev_id_count*100:.2f}%)")

payments 추가적인 app prev id 존재: 38,847
payments prev id 교집합: 958,905(57.41%)


**Defi**

In [77]:
## 둘 사이의 아귀가 맞지는 않는다
defi_id_count = len(credit_defi_id_set)
defi_inter_count = len(credit_balance_defi_id_set & credit_defi_id_set)
print(f"credit의 defi id 수: {defi_id_count:,}")
print(f"defi balance 교집합: {defi_inter_count:,}({defi_inter_count/defi_id_count*100:.2f}%)")

credit의 defi id 수: 1,716,428
defi balance 교집합: 774,354(45.11%)


In [76]:
credit_extra_count = len(credit_defi_id_set - credit_balance_defi_id_set)
balance_extra_count = len(credit_balance_defi_id_set - credit_defi_id_set)

print(f"credit에만 있는 defi id 수: {credit_extra_count:,}")
print(f"credit_balance에만 있는 defi id 수: {balance_extra_count:,}")

credit에만 있는 defi id 수: 942,074
credit_balance에만 있는 defi id 수: 43,041
